# 🧱 임베딩을 위한 준비사항

가장 중요한 것은, 수치화 과정에서 어휘를 만드는 일입니다.   
즉, 향후에 처리할 모든 단어들에 번호를 붙이는 것입니다.   
모든 단어들에 번호를 붙이고 나면 단어들을 one-hot 벡터로 바꿀 수 있게 됩니다.   
임베딩 레이어는 입력받은 번호를 one-hot 벡터로 바꾼 뒤에 가중치 행렬을 곱해서 실수 벡터로 바꾸어 줍니다.

## 🔥 **다시 한 번, 필수 루틴!**

1. 말뭉치 준비
2. 토큰 분절 (tokenization)
3. 수치화 (numericalization)
4. 배치 (batch) 구성

### 1. 말뭉치 준비

In [1]:
import pathlib
from Korpora import Korpora

this_dir = pathlib.Path().parent.resolve()
corpus = Korpora.load("nsmc", root_dir=f"{this_dir}/data").get_all_texts()
train_corpus = corpus[:-1000]
test_corpus = corpus[-1000:]


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:01, 11.3MB/s]                     
[nsmc] download ratings_test.txt: 4.90MB [00:00, 10.4MB/s]                      


### 2. 토큰 분절

In [ ]:
#!pip install torchtext==0.5.0

In [2]:
from torchtext.data import Dataset
from torchtext.data import Example
from tqdm import tqdm


class TokenizedDataset(Dataset):

    def __init__(self, corpus, processors):
        examples = []
        for example in tqdm(corpus, desc="정답 생성 및 토큰 분절 중입니다"):
            text_pair = (example, example)
            examples.append(Example.fromlist(text_pair, processors))
        super().__init__(examples, processors)

In [3]:
from sentencepiece import SentencePieceProcessor
from sentencepiece import SentencePieceTrainer

# Subword 토큰 분절 알고리즘(unigram language model)을 사용합니다.
SentencePieceTrainer.train(input=f"{this_dir}/data/nsmc/ratings_train.txt", model_prefix="spm",
						   vocab_size=10000)
tokenizer = SentencePieceProcessor(model_file="./spm.model")

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /home/piai/Using_Git_Directory/Hustar_Study_Document/NLP/data/nsmc/ratings_train.txt
  input_format: 
  model_prefix: spm
  model_type: UNIGRAM
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 

In [4]:
from torchtext.data import Field

processors = list()
bpe_func = lambda raw_text: tokenizer.encode(' '.join(raw_text), out_type=str)
processors.append(("src", Field(sequential=True, use_vocab=True,
			  	                init_token="<bos>", preprocessing=bpe_func,
								pad_token="<pad>", unk_token="<unk>",
								batch_first=True)))
processors.append(("tgt", Field(sequential=True, use_vocab=True,
                                init_token=None,
								eos_token="<eos>", preprocessing=bpe_func,
								pad_token="<pad>", unk_token="<unk>",
								batch_first=True)))

train_dataset = TokenizedDataset(train_corpus, processors)
test_dataset = TokenizedDataset(test_corpus, processors)

 sub_iter=0 size=11994 obj=16.336 num_tokens=1706845 num_tokens/piece=142.308
unigram_model_trainer.cc(507) LOG(INFO) EM sub_iter=1 size=11994 obj=16.2339 num_tokens=1706899 num_tokens/piece=142.313
unigram_model_trainer.cc(507) LOG(INFO) EM sub_iter=0 size=11000 obj=16.3661 num_tokens=1731061 num_tokens/piece=157.369
unigram_model_trainer.cc(507) LOG(INFO) EM sub_iter=1 size=11000 obj=16.3367 num_tokens=1731079 num_tokens/piece=157.371
trainer_interface.cc(685) LOG(INFO) Saving model: spm.model
trainer_interface.cc(697) LOG(INFO) Saving vocabs: spm.vocab
정답 생성 및 토큰 분절 중입니다: 100%|█| 199000/199000 [00:10<00:00, 19720.93it/
정답 생성 및 토큰 분절 중입니다: 100%|███| 1000/1000 [00:00<00:00, 20900.98it/s]


### 3. 수치화

In [5]:
for processor in processors:
    processor[1].build_vocab(train_dataset)

![embedding_dimension](https://www.tensorflow.org/text/guide/images/embedding2.png)

이제 임베딩 레이어를 만들어 봅시다.   
위의 그림을 구현하려면 "*embedding_dim*" 파라미터에 4를 주면 됩니다.

In [6]:
import torch.nn as nn

emb = nn.Embedding(len(processors[0][1].vocab), embedding_dim=4)

임베딩 레이어의 핵심은 **가중치 행렬**입니다.   
$ \text{(one-hot 벡터)} \times \text{(가중치 행렬)} = \text{(임베디드 벡터)} $이기 때문입니다.   
그렇다면 가중치 행렬을 출력해 봅시다.

In [7]:
print(emb.weight)

Parameter containing:
tensor([[-1.3231,  1.9964, -0.9644, -0.2543],
        [ 1.7903,  0.6064,  0.4292,  0.4127],
        [ 1.8166, -0.6263,  0.4808, -0.5154],
        ...,
        [ 0.4514, -0.0170, -1.8670,  0.0587],
        [ 1.9650, -0.3014, -0.3282, -0.4154],
        [-0.0903,  0.5793, -0.4691,  0.7717]], requires_grad=True)


### 4. 배치 구성

이제 배치를 구성하고 해당 배치 내 단어의 임베딩 벡터를 출력해 봅시다.

In [8]:
from torchtext.data import Iterator

BATCH_SIZE = 60

train_batches = Iterator(train_dataset, batch_size=BATCH_SIZE,
						 repeat=False, shuffle=True, sort=False)
test_batches = Iterator(test_dataset, batch_size=BATCH_SIZE,
						repeat=False, shuffle=False, sort=False)

for i, batch in enumerate(test_batches):
	print(f"임베딩이 완료된 텐서의 사이즈: {emb(batch.src).shape}")
	break

임베딩이 완료된 텐서의 사이즈: torch.Size([60, 63, 4])


In [9]:
import torch

# CUDA API를 통해 GPU를 사용할 수 있는지 확인하고 torch.device()로 장치 이름을 가져옵니다.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:

import torch.nn as nn


class RNNModel(nn.Module): 
    """
    RNN 언어 모델 클래스 (PyTorch의 nn.Module을 상속받아서
    신경망을 정의합니다.)
    """

    def __init__(self, rep_dim, n_layers, vocab_size):
        super().__init__()

        # 모델에서 사용할 하이퍼파라미터를 선언합니다.
        self.rep_dim = rep_dim
        self.n_layers = n_layers
        self.vocab_size = vocab_size

        # 레이어를 생성합니다.
        self.emb = nn.Embedding(vocab_size, embedding_dim=rep_dim)
        # torch.size([배치 내 시퀀스 개수, 시퀀스 (최대) 길이, 은닉 표상의 차원])
        self.rnn = nn.LSTM(rep_dim, rep_dim, n_layers, batch_first=True)
        self.out = nn.ModuleList([nn.Linear(rep_dim, vocab_size),
                                     nn.LogSoftmax(dim=-1)])
        
        # (학습할) 파라미터를 초기화합니다.
        self.init_weights()

    def init_weights(self):
        """ 레이어 안의 파라미터를 초기화합니다. """
        init_range = 0.1
        self.emb.weight.data.uniform_(-init_range, init_range)
        self.out[0].weight.data.uniform_(-init_range, init_range)
        self.out[0].bias.data.zero_()

    def forward(self, inp, init_hid_rep): 
        """ 모델 호출시 실행됩니다. 
        반환값:
            각 타임 스텝의 RNN 출력.
        """
        # torch.size([배치 내 시퀀스 개수, 시퀀스 (최대) 길이, 은닉 표상의 차원])
        embedded = self.emb(inp)
        
        # ``self.rnn``은 각 타임 스텝의 최종 레이어의 표상과
        # (torch.size[배치 내 시퀀스 개수, 시퀀스 (최대) 길이, 은닉 표상의 차원]))
        # 레이어별 최종 타임 스텝의 은닉 표상을
        # (torch.size[레이어 개수, 배치 내 시퀀스 개수, 은닉 표상의 차원]))
        # 반환합니다.
        last_layer_rep, last_hid_rep = self.rnn(embedded, init_hid_rep)
        
        # torch.size([(배치 내 시퀀스 개수 * 시퀀스 (최대) 길이), 은닉 표상의 차원])
        prob = self.out[0](last_layer_rep).view(-1, self.vocab_size)
        log_prob = self.out[1](prob)

        return log_prob, last_hid_rep

    def get_initial_hid_rep(self, batch_size):
        # 파라미터를 아무거나 하나 가져옵니다.
        weight = next(self.parameters())
        # ``new_zeros()`` 메소드는 해당 파라미터와 동일한
        # `torch.dtype`과 `torch.device` 값을 가지는,
        # 0으로 채워진 torch.Tensor를 생성합니다.
        h_0 = weight.new_zeros(self.n_layers, batch_size, self.rep_dim)
        c_0 = weight.new_zeros(self.n_layers, batch_size, self.rep_dim)
        return h_0, c_0

In [11]:
# 하이퍼파라미터를 정의합니다.
REP_DIM = 256
MAX_EPOCH = 1

# 모델을 생성합니다.
vocab_size = len(processors[0][1].vocab)
model = RNNModel(REP_DIM, n_layers=2, vocab_size=vocab_size).to(device)

# 학습에 사용할 최적화 기법과 손실 함수를 정의합니다.
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.NLLLoss(ignore_index=0, reduction='mean')

In [12]:
def cal_acc(scores, target):
    """ 모델 예측의 정확도(%)를 계산합니다. """
    pred = scores.max(-1)[1]
    non_pad = target.ne(0)
    num_correct = pred.eq(target).masked_select(non_pad).sum().item() 
    num_non_pad = non_pad.sum().item()
    return 100 * (num_correct / num_non_pad)

In [13]:
import statistics
import time

from tqdm import tqdm


def train():
    model.train()
    mean_loss = []
    mean_acc = []
    start_time = time.time()
    for batch in tqdm(train_batches, desc="훈련 중입니다."):
        src = batch.src.to(device)
        tgt = batch.tgt.view(-1).to(device)
        init_hid_rep = []
        for hid_rep in model.get_initial_hid_rep(len(batch)):
            init_hid_rep.append(hid_rep.to(device))
        optimizer.zero_grad()
        log_prob, last_hid_rep = model(src, init_hid_rep)
        # 손실값을 역전파합니다.
        loss = criterion(log_prob, tgt)
        loss.backward()
        optimizer.step()
        # 평가를 위해서 결과를 기록합니다.
        mean_loss.append(loss.item())
        mean_acc.append(cal_acc(log_prob, tgt))
    total_time = time.time() - start_time
    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)
    return mean_loss, total_time, mean_acc


def evaluate():
    model.eval()
    mean_loss = []
    mean_acc = []
    for batch in tqdm(test_batches, desc="평가 중입니다."):
        src = batch.src.to(device)
        tgt = batch.tgt.view(-1).to(device)
        init_hid_rep = []
        for hid_rep in model.get_initial_hid_rep(len(batch)):
            init_hid_rep.append(hid_rep.to(device))
        with torch.no_grad():
            log_prob, last_hid_rep = model(src, init_hid_rep)
            loss = criterion(log_prob, tgt)
            mean_loss.append(loss.item())
            mean_acc.append(cal_acc(log_prob, tgt))
    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)
    return mean_loss, mean_acc

In [14]:
for epoch in range(1, MAX_EPOCH + 1):
    train_batches = Iterator(train_dataset, batch_size=BATCH_SIZE,
                             repeat=False, shuffle=True, sort=False)
    loss, elapsed_time, accuracy = train()
    print('epoch {:4d} | times {:3.3f} |  loss: {:3.3f} | accuracy: {:3.2f}'.format(epoch, elapsed_time, loss, accuracy))

    if epoch % 10 == 0:
        loss, accuracy = evaluate()
        print('=' * 60)
        print('Evaluation | loss: {:3.3f} | accuracy: {:3.2f}'.format(loss, accuracy))
        print('=' * 60)

with open('model.pt', 'wb') as f:
    print('save model at: ./model.pt')
    torch.save(model, f)

훈련 중입니다.: 100%|███████████████████████| 3317/3317 [01:41<00:00, 32.53it/s]

epoch    1 | times 101.962 |  loss: 1.598 | accuracy: 77.30
save model at: ./model.pt


In [15]:
import numpy as np


def pred_seq_prob(seq):
    model.eval()
    with torch.no_grad():
        # Question 1: 정답 데이터 프로세싱
        src_seq = []
        src_seq.append(processors[0][1].vocab.stoi["<bos>"])
        for src_tok in processors[0][1].preprocess(seq):
            src_seq.append(processors[0][1].vocab.stoi[src_tok])
        src = torch.tensor(src_seq, dtype=torch.long)
        
        tgt_seq = []
        # ?
        tgt_seq.append(processors[1][1].vocab.stoi["<eos>"])

        # 은닉 표상 초기화
        init_hid_rep = []
        for hid_rep in model.get_initial_hid_rep(1):
            init_hid_rep.append(hid_rep.squeeze(1))

        # Question 2: 로그 확률 산출
        # ?

        # 토큰별 로그 확률 산출
        tgt = tgt.unsqueeze(-1)
        tok_probabilities = torch.gather(log_prob, 1, tgt).tolist()

        # Question 3. 로그 확률의 합을 결과로 반환
        # return ?

# load saved model
with open('./model.pt', 'rb') as f:
    print('load model from: ./model.pt')
    model = torch.load(f).to("cpu")
    print('``나는 불쌍한 대학원생이에요.``: {:3.3f}'.format(pred_seq_prob("나는 불쌍한 대학원생이에요.")))
    print('``나는 부유한 대학원생이에요.``: {:3.3f}'.format(pred_seq_prob("나는 부유한 대학원생이에요.")))

load model from: ./model.pt


UnboundLocalError: local variable 'tgt' referenced before assignment